In [2]:
import json

# एक JSON स्ट्रिंग (मजकूर)
json_string = '''
{
  "student_name": "Priya Deshmukh",
  "age": 21,
  "is_enrolled": true,
  "courses": [
    {"course_name": "Data Science", "credits": 4},
    {"course_name": "Machine Learning", "credits": 4}
  ]
}
'''

# JSON स्ट्रिंगला पायथॉन डिक्शनरीमध्ये रूपांतरित करणे
data_dictionary = json.loads(json_string)

print("Type of the converted data:", type(data_dictionary))
print("Student's name:", data_dictionary['student_name'])
print("First course name:", data_dictionary['courses'][0]['course_name'])

# पायथॉन डिक्शनरीला परत JSON स्ट्रिंगमध्ये रूपांतरित करणे
new_json_string = json.dumps(data_dictionary, indent=4) # indent=4 मुळे ते सुंदर दिसते
print("\nConverted back to JSON string:\n", new_json_string)

Type of the converted data: <class 'dict'>
Student's name: Priya Deshmukh
First course name: Data Science

Converted back to JSON string:
 {
    "student_name": "Priya Deshmukh",
    "age": 21,
    "is_enrolled": true,
    "courses": [
        {
            "course_name": "Data Science",
            "credits": 4
        },
        {
            "course_name": "Machine Learning",
            "credits": 4
        }
    ]
}


In [3]:
import pandas as pd
# CSV फाइल वाचणे
# ही फाइल तुमच्या संगणकावर त्याच फोल्डरमध्ये असली पाहिजे जिथे तुम्ही कोड चालवत आहात.
try:
    books_df = pd.read_csv('books_data.csv')
    print("Successfully read the CSV file:\n")
    print(books_df)
except FileNotFoundError:
    print("Error: books_data.csv not found. Please create the file with the sample data.")

Successfully read the CSV file:

              Book_Title             Author      Genre  Price  Rating
0          The Alchemist       Paulo Coelho  Adventure    250     4.6
1                Sapiens  Yuval Noah Harari    History    400     4.7
2                   1984      George Orwell  Dystopian    300     4.8
3  To Kill a Mockingbird         Harper Lee    Fiction    350     4.5


In [4]:
# एक्सेल फाइल वाचणे
# यासाठी openpyxl लायब्ररी आवश्यक आहे
# आणि financial_data.xlsx नावाची फाइल तयार असणे आवश्यक आहे.

# उदाहरणासाठी आपण आधी फाइल तयार करूया
writer = pd.ExcelWriter('financial_data.xlsx', engine='openpyxl')

revenue_df = pd.DataFrame({'Year': [2022, 2023, 2024], 'Revenue_Cr': [500, 550, 620]})
revenue_df.to_excel(writer, sheet_name='Revenue', index=False)

expenses_df = pd.DataFrame({'Year': [2022, 2023, 2024], 'Expenses_Cr': [350, 380, 410]})
expenses_df.to_excel(writer, sheet_name='Expenses', index=False)

writer.close() # close() ऐवजी save() जुन्या आवृत्त्यांमध्ये होते, पण close() वापरणे चांगले.

# आता 'Revenue' शीट वाचूया
revenue_data = pd.read_excel('financial_data.xlsx', sheet_name='Revenue')
print("Data from the 'Revenue' sheet:\n")
print(revenue_data)

# सर्व शीट्स एकाच वेळी वाचणे
all_sheets = pd.read_excel('financial_data.xlsx', sheet_name=None)
print("\nType of 'all_sheets':", type(all_sheets))
print("Name of the sheets:", all_sheets.keys())

# 'Expenses' शीटमधील डेटा पाहणे
print("\nData from the 'Expenses' sheet:\n")
print(all_sheets['Expenses'])

Data from the 'Revenue' sheet:

   Year  Revenue_Cr
0  2022         500
1  2023         550
2  2024         620

Type of 'all_sheets': <class 'dict'>
Name of the sheets: dict_keys(['Revenue', 'Expenses'])

Data from the 'Expenses' sheet:

   Year  Expenses_Cr
0  2022          350
1  2023          380
2  2024          410


In [5]:
import requests

# एका वेबसाइटचा पत्ता
url = 'https://books.toscrape.com/'

# त्या पत्त्यावर GET विनंती पाठवणे
try:
    response = requests.get(url)

    # विनंती यशस्वी झाली का हे तपासणे
    # Status code 200 म्हणजे 'OK' किंवा यशस्वी
    if response.status_code == 200:
        print("Successfully fetched the webpage!")
        # पानाची सामग्री (HTML) पाहणे (फक्त पहिले ५०० अक्षर)
        # print(response.text[:500])
    else:
        print(f"Failed to fetch webpage. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Successfully fetched the webpage!


In [6]:
from bs4 import BeautifulSoup

# आधीच्या उदाहरणातील response.text वापरूया
# response.text मध्ये HTML कोड आहे असे गृहीत धरूया

html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

# सूप ऑब्जेक्ट तयार झाला आहे
print("\nType of soup object:", type(soup))
# वेबसाइटचे शीर्षक मिळवणे
print("Website Title:", soup.title.get_text())


Type of soup object: <class 'bs4.BeautifulSoup'>
Website Title: 
    All products | Books to Scrape - Sandbox



In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = 'https://books.toscrape.com/'
scraped_data = []

print("Starting the scraper...")

try:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # सर्व पुस्तकांना सामावणारे 'article' टॅग्ज शोधणे
        books = soup.find_all('article', class_='product_pod')
        
        print(f"Found {len(books)} books on the page.")
        
        # प्रत्येक पुस्तकावरून माहिती काढण्यासाठी लूप चालवणे
        for book in books:
            # पुस्तकाचे नाव मिळवणे
            # नाव h3 टॅगच्या आत a टॅगमध्ये आहे
            title_tag = book.find('h3').find('a')
            title = title_tag['title'] if title_tag else 'No Title Found'
            
            # पुस्तकाची किंमत मिळवणे
            price_tag = book.find('p', class_='price_color')
            price_text = price_tag.get_text() if price_tag else '£0.00'
            # '£' चिन्ह काढून टाकून फक्त संख्या घेणे
            price = float(price_text.replace('Â£',''))
            
            # पुस्तकाचे रेटिंग मिळवणे
            # रेटिंग p टॅगच्या क्लास नावात आहे, उदा. 'star-rating Three'
            rating_tag = book.find('p', class_='star-rating')
            rating_classes = rating_tag['class'] if rating_tag else [] # ['star-rating', 'Three']
            # रेटिंग शब्दाला संख्यात्मक रूपात बदलणे
            rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
            rating_word = rating_classes[1] if len(rating_classes) > 1 else None
            rating = rating_map.get(rating_word, 0)
            
            # काढलेली माहिती एका डिक्शनरीमध्ये साठवणे
            scraped_data.append({
                'title': title,
                'price_in_pounds': price,
                'rating_out_of_5': rating
            })
            
        print("Scraping finished successfully.")

    else:
        print(f"Failed to fetch page. Status Code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

# स्क्रॅप केलेल्या डेटाला एका Pandas डेटाफ्रेममध्ये रूपांतरित करणे
if scraped_data:
    books_scraped_df = pd.DataFrame(scraped_data)
    
    print("\n--- Scraped Data ---")
    print(books_scraped_df.head()) # फक्त पहिले ५ पुस्तके दाखवणे
    
    # डेटाफ्रेमला CSV फाइलमध्ये सेव्ह करणे
    # books_scraped_df.to_csv('scraped_books.csv', index=False)
    # print("\nData saved to scraped_books.csv")

Starting the scraper...
Found 20 books on the page.
Scraping finished successfully.

--- Scraped Data ---
                                   title  price_in_pounds  rating_out_of_5
0                   A Light in the Attic            51.77                3
1                     Tipping the Velvet            53.74                1
2                             Soumission            50.10                1
3                          Sharp Objects            47.82                4
4  Sapiens: A Brief History of Humankind            54.23                5


In [9]:
price_text

'Â£51.77'

In [12]:
import requests
import json # JSON सुंदर दिसण्यासाठी

# ISS च्या सध्याच्या जागेसाठीचा API एंडपॉइंट
url = "http://api.open-notify.org/iss-now.json"

print(f"Requesting data from: {url}")

try:
    response = requests.get(url)
    
    # विनंती यशस्वी झाली का हे तपासणे
    if response.status_code == 200:
        print("API request successful!")
        
        # मिळालेल्या JSON माहितीला पायथॉन डिक्शनरीमध्ये रूपांतरित करणे
        # response.json() हे काम आपल्यासाठी सोपे करते
        data = response.json()
        
        print("\n--- Raw Data Received ---")
        # मिळालेला डेटा सुंदर स्वरूपात प्रिंट करणे
        print(json.dumps(data, indent=4))
        
        # डिक्शनरीमधून आवश्यक माहिती काढणे
        timestamp = data['timestamp']
        latitude = data['iss_position']['latitude']
        longitude = data['iss_position']['longitude']
        
        print("\n--- Processed Information ---")
        print(f"At timestamp {timestamp}, the ISS is currently at:")
        print(f"Latitude: {latitude}")
        print(f"Longitude: {longitude}")
        
    else:
        print(f"API request failed with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Requesting data from: http://api.open-notify.org/iss-now.json
API request successful!

--- Raw Data Received ---
{
    "iss_position": {
        "longitude": "89.8150",
        "latitude": "-30.6721"
    },
    "timestamp": 1757325360,
    "message": "success"
}

--- Processed Information ---
At timestamp 1757325360, the ISS is currently at:
Latitude: -30.6721
Longitude: 89.8150


In [13]:
import requests
import pandas as pd

# अवकाशातील लोकांची माहिती देणारा एंडपॉइंट
url_astros = "http://api.open-notify.org/astros.json"

print(f"Requesting data from: {url_astros}")

try:
    response = requests.get(url_astros)
    if response.status_code == 200:
        data = response.json()
        
        number_of_people = data['number']
        print(f"\nThere are currently {number_of_people} people in space.")
        
        # अंतराळवीरांची यादी मिळवणे
        astronauts = data['people']
        
        # यादीला एका सुंदर Pandas डेटाफ्रेममध्ये रूपांतरित करणे
        astronauts_df = pd.DataFrame(astronauts)
        
        print("\n--- List of Astronauts ---")
        print(astronauts_df)
        
    else:
        print(f"API request failed with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Requesting data from: http://api.open-notify.org/astros.json

There are currently 12 people in space.

--- List of Astronauts ---
       craft                  name
0        ISS        Oleg Kononenko
1        ISS          Nikolai Chub
2        ISS  Tracy Caldwell Dyson
3        ISS      Matthew Dominick
4        ISS       Michael Barratt
5        ISS         Jeanette Epps
6        ISS   Alexander Grebenkin
7        ISS         Butch Wilmore
8        ISS       Sunita Williams
9   Tiangong            Li Guangsu
10  Tiangong               Li Cong
11  Tiangong            Ye Guangfu
